# Sistema de Compliance LGPD

## Demonstração - Auditoria e Anonimização de Dados Pessoais

Este notebook demonstra as funcionalidades do sistema de compliance:

1. **Scanner de PII**: Detecta dados pessoais identificáveis
2. **Classificação de Risco**: Categoriza dados por nível de sensibilidade
3. **Anonimização**: 7 métodos diferentes de proteção
4. **Relatórios**: Documentação para auditoria

### Referências LGPD
- **Art. 5º**: Definição de dados pessoais e sensíveis
- **Art. 6º**: Princípios (finalidade, necessidade, transparência)
- **Art. 12**: Anonimização como alternativa ao consentimento

---

In [ ]:
# Instalar dependências (necessário no Google Colab)
!pip install faker -q

# Imports
import pandas as pd
import numpy as np
from faker import Faker
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import re
import hashlib
from enum import Enum
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Any
from datetime import datetime

warnings.filterwarnings('ignore')

# ============================================================
# CLASSES DO SISTEMA DE COMPLIANCE (versão simplificada para demo)
# ============================================================

class PIIType(Enum):
    """Tipos de dados pessoais identificáveis"""
    CPF = "cpf"
    CNPJ = "cnpj"
    EMAIL = "email"
    PHONE = "telefone"
    NAME = "nome"
    ADDRESS = "endereco"
    DATE_BIRTH = "data_nascimento"
    SALARY = "salario"
    CEP = "cep"
    GENERIC = "generico"

class RiskLevel(Enum):
    """Níveis de risco LGPD"""
    CRITICO = "critico"
    ALTO = "alto"
    MEDIO = "medio"
    BAIXO = "baixo"

class AnonymizationMethod(Enum):
    """Métodos de anonimização disponíveis"""
    MASK = "mask"
    HASH = "hash"
    PSEUDONYMIZE = "pseudonymize"
    GENERALIZE = "generalize"
    SUPPRESS = "suppress"
    TOKENIZE = "tokenize"
    NOISE = "noise"

@dataclass
class PIIDetection:
    """Resultado de detecção de PII"""
    column: str
    pii_type: PIIType
    risk_level: RiskLevel
    detection_method: str
    count: int
    percentage: float
    sample_values: List[str] = field(default_factory=list)

@dataclass
class ScanResult:
    """Resultado completo do scan"""
    source_name: str
    total_rows: int
    columns_scanned: int
    pii_found: List[PIIDetection]
    risk_summary: Dict[str, int]
    recommendations: List[str]
    scan_duration_seconds: float

class PIIScanner:
    """Scanner de dados pessoais identificáveis"""
    
    PATTERNS = {
        PIIType.CPF: r'\d{3}\.?\d{3}\.?\d{3}-?\d{2}',
        PIIType.CNPJ: r'\d{2}\.?\d{3}\.?\d{3}/?\d{4}-?\d{2}',
        PIIType.EMAIL: r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}',
        PIIType.PHONE: r'(\(?\d{2}\)?\s?)?(\d{4,5}-?\d{4})',
        PIIType.CEP: r'\d{5}-?\d{3}',
    }
    
    COLUMN_KEYWORDS = {
        'nome': (PIIType.NAME, RiskLevel.ALTO),
        'name': (PIIType.NAME, RiskLevel.ALTO),
        'cpf': (PIIType.CPF, RiskLevel.CRITICO),
        'cnpj': (PIIType.CNPJ, RiskLevel.CRITICO),
        'email': (PIIType.EMAIL, RiskLevel.ALTO),
        'telefone': (PIIType.PHONE, RiskLevel.MEDIO),
        'phone': (PIIType.PHONE, RiskLevel.MEDIO),
        'endereco': (PIIType.ADDRESS, RiskLevel.MEDIO),
        'address': (PIIType.ADDRESS, RiskLevel.MEDIO),
        'nascimento': (PIIType.DATE_BIRTH, RiskLevel.MEDIO),
        'birth': (PIIType.DATE_BIRTH, RiskLevel.MEDIO),
        'salario': (PIIType.SALARY, RiskLevel.ALTO),
        'salary': (PIIType.SALARY, RiskLevel.ALTO),
        'cep': (PIIType.CEP, RiskLevel.BAIXO),
    }
    
    def scan(self, df: pd.DataFrame, source_name: str = "unknown") -> ScanResult:
        start_time = datetime.now()
        pii_found = []
        
        for col in df.columns:
            col_lower = col.lower()
            
            # Verificar por nome da coluna
            for keyword, (pii_type, risk) in self.COLUMN_KEYWORDS.items():
                if keyword in col_lower:
                    sample = df[col].dropna().head(3).astype(str).tolist()
                    pii_found.append(PIIDetection(
                        column=col,
                        pii_type=pii_type,
                        risk_level=risk,
                        detection_method="keyword_match",
                        count=df[col].notna().sum(),
                        percentage=round(df[col].notna().sum() / len(df) * 100, 1),
                        sample_values=sample
                    ))
                    break
            else:
                # Verificar por padrão regex
                if df[col].dtype == 'object':
                    for pii_type, pattern in self.PATTERNS.items():
                        matches = df[col].astype(str).str.contains(pattern, regex=True, na=False)
                        if matches.sum() > len(df) * 0.5:
                            risk = RiskLevel.CRITICO if pii_type in [PIIType.CPF, PIIType.CNPJ] else RiskLevel.MEDIO
                            sample = df[col][matches].head(3).astype(str).tolist()
                            pii_found.append(PIIDetection(
                                column=col,
                                pii_type=pii_type,
                                risk_level=risk,
                                detection_method="pattern_match",
                                count=matches.sum(),
                                percentage=round(matches.sum() / len(df) * 100, 1),
                                sample_values=sample
                            ))
                            break
        
        risk_summary = {}
        for pii in pii_found:
            risk_summary[pii.risk_level.value] = risk_summary.get(pii.risk_level.value, 0) + 1
        
        recommendations = self._generate_recommendations(pii_found)
        duration = (datetime.now() - start_time).total_seconds()
        
        return ScanResult(
            source_name=source_name,
            total_rows=len(df),
            columns_scanned=len(df.columns),
            pii_found=pii_found,
            risk_summary=risk_summary,
            recommendations=recommendations,
            scan_duration_seconds=round(duration, 3)
        )
    
    def _generate_recommendations(self, pii_found: List[PIIDetection]) -> List[str]:
        recs = []
        risk_levels = [p.risk_level for p in pii_found]
        
        if RiskLevel.CRITICO in risk_levels:
            recs.append("⚠️ URGENTE: Dados de risco CRÍTICO detectados. Aplicar anonimização imediata.")
        if RiskLevel.ALTO in risk_levels:
            recs.append("🔴 Dados de risco ALTO requerem consentimento explícito ou anonimização.")
        if any(p.pii_type == PIIType.CPF for p in pii_found):
            recs.append("📋 CPFs detectados devem ser armazenados com criptografia ou hash.")
        if any(p.pii_type == PIIType.SALARY for p in pii_found):
            recs.append("💰 Dados salariais são sensíveis - considerar generalização em faixas.")
        recs.append("📝 Documentar finalidade e base legal para tratamento de cada tipo de dado.")
        recs.append("🔒 Implementar controle de acesso baseado em necessidade (need-to-know).")
        
        return recs

class DataAnonymizer:
    """Anonimizador de dados com múltiplos métodos"""
    
    def __init__(self):
        self.fake = Faker('pt_BR')
        self.token_mapping = {}
        self.salt = "lgpd_demo_2024"
    
    def clear_token_mapping(self):
        self.token_mapping = {}
    
    def anonymize_column(self, df: pd.DataFrame, column: str, method: AnonymizationMethod, **kwargs) -> pd.DataFrame:
        df = df.copy()
        
        if method == AnonymizationMethod.MASK:
            df[column] = df[column].apply(lambda x: self._mask(str(x), **kwargs))
        elif method == AnonymizationMethod.HASH:
            df[column] = df[column].apply(lambda x: self._hash(str(x), **kwargs))
        elif method == AnonymizationMethod.PSEUDONYMIZE:
            df[column] = df[column].apply(lambda x: self._pseudonymize(**kwargs))
        elif method == AnonymizationMethod.GENERALIZE:
            df[column] = self._generalize(df[column], **kwargs)
        elif method == AnonymizationMethod.SUPPRESS:
            df[column] = kwargs.get('replacement', '[SUPRIMIDO]')
        elif method == AnonymizationMethod.TOKENIZE:
            df[column] = df[column].apply(lambda x: self._tokenize(str(x), **kwargs))
        elif method == AnonymizationMethod.NOISE:
            df[column] = df[column].apply(lambda x: self._add_noise(x, **kwargs))
        
        return df
    
    def _mask(self, value: str, visible_start: int = 3, visible_end: int = 2, **kwargs) -> str:
        if len(value) <= visible_start + visible_end:
            return '*' * len(value)
        masked = value[:visible_start] + '*' * (len(value) - visible_start - visible_end)
        if visible_end > 0:
            masked += value[-visible_end:]
        return masked
    
    def _hash(self, value: str, truncate: int = 16, **kwargs) -> str:
        salted = f"{self.salt}{value}"
        hashed = hashlib.sha256(salted.encode()).hexdigest()
        return hashed[:truncate]
    
    def _pseudonymize(self, pii_type: str = 'name', **kwargs) -> str:
        if pii_type == 'name':
            return self.fake.name()
        elif pii_type == 'email':
            return self.fake.email()
        elif pii_type == 'cpf':
            return self.fake.cpf()
        return self.fake.word()
    
    def _generalize(self, series: pd.Series, bins: int = 4, labels: List[str] = None, **kwargs) -> pd.Series:
        if pd.api.types.is_numeric_dtype(series):
            if labels is None:
                labels = [f'Faixa {i+1}' for i in range(bins)]
            return pd.cut(series, bins=bins, labels=labels)
        return series.apply(lambda x: str(x)[:4] + '...' if len(str(x)) > 4 else str(x))
    
    def _tokenize(self, value: str, prefix: str = 'TOK_', **kwargs) -> str:
        if value not in self.token_mapping:
            self.token_mapping[value] = f"{prefix}{len(self.token_mapping):05d}"
        return self.token_mapping[value]
    
    def _add_noise(self, value: Any, noise_level: float = 0.1, **kwargs) -> Any:
        if isinstance(value, (int, float)):
            noise = np.random.uniform(-noise_level, noise_level) * value
            return round(value + noise, 2)
        return value
    
    def anonymize_dataframe(self, df: pd.DataFrame, config: Dict[str, Dict]) -> pd.DataFrame:
        df = df.copy()
        for column, params in config.items():
            if column in df.columns:
                method = AnonymizationMethod(params.pop('method'))
                df = self.anonymize_column(df, column, method, **params)
                params['method'] = method.value  # Restaurar para config
        return df

# Cores do tema
CORES = {
    'critico': '#DC3545',
    'alto': '#FD7E14',
    'medio': '#FFC107',
    'baixo': '#28A745',
    'primaria': '#2E86AB',
    'secundaria': '#A23B72'
}

print('✅ Sistema de Compliance LGPD carregado!')

## 1. Gerando Dados de Exemplo

Vamos criar um dataset fictício que simula dados de clientes com vários tipos de PII (Personally Identifiable Information).

In [ ]:
# Gerar dados fictícios
fake = Faker('pt_BR')
Faker.seed(42)
np.random.seed(42)

n_registros = 100

df_clientes = pd.DataFrame({
    'id': range(1, n_registros + 1),
    'nome_completo': [fake.name() for _ in range(n_registros)],
    'cpf': [fake.cpf() for _ in range(n_registros)],
    'email': [fake.email() for _ in range(n_registros)],
    'telefone': [fake.phone_number() for _ in range(n_registros)],
    'data_nascimento': [fake.date_of_birth(minimum_age=18, maximum_age=80).strftime('%d/%m/%Y') for _ in range(n_registros)],
    'endereco': [fake.street_address() for _ in range(n_registros)],
    'cidade': [fake.city() for _ in range(n_registros)],
    'cep': [fake.postcode() for _ in range(n_registros)],
    'salario': np.random.uniform(1500, 25000, n_registros).round(2),
    'cargo': np.random.choice(['Analista', 'Gerente', 'Diretor', 'Assistente', 'Coordenador'], n_registros),
    'departamento': np.random.choice(['TI', 'RH', 'Financeiro', 'Comercial', 'Operações'], n_registros),
    'observacao': ['Cadastro regular' for _ in range(n_registros)]
})

print(f'📊 Dataset gerado: {len(df_clientes)} registros, {len(df_clientes.columns)} colunas')
print(f'\nColunas: {list(df_clientes.columns)}')
df_clientes.head()

## 2. Scanner de PII - Detectando Dados Pessoais

O scanner analisa cada coluna em busca de:
- **Padrões regex**: CPF, CNPJ, e-mail, telefone, etc.
- **Nomes de colunas**: "nome", "endereco", "salario", etc.
- **Características estatísticas**: Formatos típicos de PII

In [ ]:
# Inicializar scanner
scanner = PIIScanner()

# Executar scan
resultado = scanner.scan(df_clientes, source_name='dados_clientes.csv')

print('=' * 70)
print('RESULTADO DO SCAN DE DADOS PESSOAIS')
print('=' * 70)
print(f'\n📄 Arquivo: {resultado.source_name}')
print(f'📊 Linhas analisadas: {resultado.total_rows}')
print(f'📝 Colunas analisadas: {resultado.columns_scanned}')
print(f'⚠️  PIIs encontrados: {len(resultado.pii_found)}')
print(f'⏱️  Tempo de scan: {resultado.scan_duration_seconds}s')

In [ ]:
# Detalhes dos PIIs encontrados
print('\n' + '=' * 70)
print('DETALHES DOS DADOS PESSOAIS ENCONTRADOS')
print('=' * 70)

for pii in resultado.pii_found:
    icone = {
        'critico': '🚨',
        'alto': '🔴',
        'medio': '🟡',
        'baixo': '🟢'
    }.get(pii.risk_level.value, '•')
    
    print(f'\n{icone} [{pii.risk_level.value.upper()}] {pii.column}')
    print(f'   Tipo: {pii.pii_type.value}')
    print(f'   Detecção: {pii.detection_method}')
    print(f'   Ocorrências: {pii.count} ({pii.percentage}%)')
    print(f'   Exemplos: {pii.sample_values[:2]}')

### 2.1 Visualização: Matriz de Risco

In [ ]:
# Preparar dados para visualização
df_pii = pd.DataFrame([
    {
        'coluna': pii.column,
        'tipo': pii.pii_type.value,
        'risco': pii.risk_level.value,
        'ocorrencias': pii.count,
        'metodo': pii.detection_method
    }
    for pii in resultado.pii_found
])

# Ordenar por risco
ordem_risco = ['critico', 'alto', 'medio', 'baixo']
df_pii['risco_ordem'] = df_pii['risco'].map({r: i for i, r in enumerate(ordem_risco)})
df_pii = df_pii.sort_values('risco_ordem')

# Gráfico de barras por risco
cores_risco = [CORES.get(r, '#999') for r in df_pii['risco']]

fig = px.bar(
    df_pii,
    x='coluna',
    y='ocorrencias',
    color='risco',
    color_discrete_map=CORES,
    text='tipo',
    hover_data=['metodo']
)

fig.update_layout(
    title={'text': '🚨 PIIs Detectados por Coluna e Nível de Risco', 'x': 0.5, 'font': {'size': 18}},
    xaxis_title='Coluna',
    yaxis_title='Ocorrências',
    xaxis_tickangle=-45,
    height=500,
    legend_title='Nível de Risco'
)

fig.update_traces(textposition='outside')
fig.show()

In [ ]:
# Resumo por nível de risco
resumo_risco = df_pii.groupby('risco').size().reindex(ordem_risco).fillna(0).astype(int)

fig = go.Figure(data=[go.Pie(
    labels=resumo_risco.index,
    values=resumo_risco.values,
    marker_colors=[CORES.get(r, '#999') for r in resumo_risco.index],
    hole=0.4,
    textinfo='label+value',
    textposition='outside'
)])

fig.update_layout(
    title={'text': '📊 Distribuição por Nível de Risco', 'x': 0.5, 'font': {'size': 18}},
    height=400,
    annotations=[dict(text=f'{len(resultado.pii_found)}<br>PIIs', x=0.5, y=0.5, font_size=20, showarrow=False)]
)

fig.show()

print('\n📝 Resumo de Risco:')
for nivel, count in resumo_risco.items():
    if count > 0:
        print(f'   {nivel.upper()}: {count} coluna(s)')

### 2.2 Recomendações de Compliance

In [ ]:
print('\n' + '=' * 70)
print('📜 RECOMENDAÇÕES DE COMPLIANCE (LGPD)')
print('=' * 70)

for i, rec in enumerate(resultado.recommendations, 1):
    print(f'\n{i}. {rec}')

## 3. Anonimização de Dados

O sistema oferece **7 métodos de anonimização** para proteger dados pessoais:

| Método | Descrição | Reversível? |
|--------|-----------|------------|
| `mask` | Substitui caracteres por * | Não |
| `hash` | Hash SHA-256 com salt | Não |
| `pseudonymize` | Substitui por dado falso | Não |
| `generalize` | Reduz precisão (faixas) | Não |
| `suppress` | Remove completamente | Não |
| `tokenize` | Substitui por token único | Sim |
| `noise` | Adiciona variação aleatória | Não |

In [ ]:
# Inicializar anonimizador
anonymizer = DataAnonymizer()

# Copiar dados originais
df_original = df_clientes.copy()

print('🔒 DataAnonymizer inicializado')
print('\nMétodos disponíveis:')
for method in AnonymizationMethod:
    print(f'   • {method.value}')

### 3.1 Demonstração de Cada Método

In [ ]:
# Exemplo de cada método
amostra = df_original.head(5).copy()

print('=' * 80)
print('DEMONSTRAÇÃO DOS MÉTODOS DE ANONIMIZAÇÃO')
print('=' * 80)

# 1. MASK
print('\n🎭 1. MASK (Mascaramento)')
print('-' * 40)
df_mask = anonymizer.anonymize_column(amostra.copy(), 'cpf', AnonymizationMethod.MASK, 
                                       visible_start=3, visible_end=2)
print(f'Original: {amostra["cpf"].iloc[0]}')
print(f'Masked:   {df_mask["cpf"].iloc[0]}')

# 2. HASH
print('\n#️⃣ 2. HASH (SHA-256 + Salt)')
print('-' * 40)
df_hash = anonymizer.anonymize_column(amostra.copy(), 'cpf', AnonymizationMethod.HASH, truncate=16)
print(f'Original: {amostra["cpf"].iloc[0]}')
print(f'Hashed:   {df_hash["cpf"].iloc[0]}')

# 3. PSEUDONYMIZE
print('\n👤 3. PSEUDONYMIZE (Dados Falsos)')
print('-' * 40)
df_pseudo = anonymizer.anonymize_column(amostra.copy(), 'nome_completo', AnonymizationMethod.PSEUDONYMIZE, 
                                         pii_type='name')
print(f'Original: {amostra["nome_completo"].iloc[0]}')
print(f'Pseudo:   {df_pseudo["nome_completo"].iloc[0]}')

# 4. GENERALIZE
print('\n📊 4. GENERALIZE (Faixas)')
print('-' * 40)
df_gen = anonymizer.anonymize_column(amostra.copy(), 'salario', AnonymizationMethod.GENERALIZE, 
                                      bins=4, labels=['Baixo', 'Médio', 'Alto', 'Muito Alto'])
print(f'Original: R$ {amostra["salario"].iloc[0]:,.2f}')
print(f'General.: {df_gen["salario"].iloc[0]}')

# 5. SUPPRESS
print('\n❌ 5. SUPPRESS (Remoção)')
print('-' * 40)
df_sup = anonymizer.anonymize_column(amostra.copy(), 'telefone', AnonymizationMethod.SUPPRESS, 
                                      replacement='[SUPRIMIDO]')
print(f'Original: {amostra["telefone"].iloc[0]}')
print(f'Supresso: {df_sup["telefone"].iloc[0]}')

# 6. TOKENIZE
print('\n🎫 6. TOKENIZE (Token Reversível)')
print('-' * 40)
anonymizer.clear_token_mapping()  # Limpar tokens anteriores
df_token = anonymizer.anonymize_column(amostra.copy(), 'email', AnonymizationMethod.TOKENIZE, 
                                        prefix='EMAIL_')
print(f'Original: {amostra["email"].iloc[0]}')
print(f'Token:    {df_token["email"].iloc[0]}')

# 7. NOISE
print('\n🎲 7. NOISE (Ruído Aleatório)')
print('-' * 40)
df_noise = anonymizer.anonymize_column(amostra.copy(), 'salario', AnonymizationMethod.NOISE, 
                                        noise_level=0.1)
print(f'Original: R$ {amostra["salario"].iloc[0]:,.2f}')
print(f'Com Ruído: R$ {df_noise["salario"].iloc[0]:,.2f}')

### 3.2 Anonimização Completa do Dataset

In [ ]:
# Configuração de anonimização por coluna
config_anonimizacao = {
    'nome_completo': {'method': 'pseudonymize', 'pii_type': 'name'},
    'cpf': {'method': 'hash', 'truncate': 12},
    'email': {'method': 'mask', 'visible_start': 2, 'visible_end': 0},
    'telefone': {'method': 'mask', 'pattern': '(**) *****-****'},
    'data_nascimento': {'method': 'generalize', 'generalization_type': 'truncate'},
    'endereco': {'method': 'suppress', 'replacement': '[PROTEGIDO]'},
    'cep': {'method': 'mask', 'visible_start': 5, 'visible_end': 0},
    'salario': {'method': 'generalize', 'bins': 5, 'labels': ['Até 3k', '3k-6k', '6k-10k', '10k-15k', 'Acima 15k']}
}

print('📝 Configuração de Anonimização:')
for col, params in config_anonimizacao.items():
    print(f'   {col}: {params["method"]}')

# Aplicar anonimização
anonymizer_full = DataAnonymizer()
df_anonimizado = anonymizer_full.anonymize_dataframe(df_original.copy(), config_anonimizacao)

print(f'\n✅ Dataset anonimizado com sucesso!')

### 3.3 Comparação: Antes vs Depois

In [ ]:
# Comparar antes e depois
print('=' * 100)
print('COMPARAÇÃO: DADOS ORIGINAIS vs ANONIMIZADOS')
print('=' * 100)

colunas_comparar = ['nome_completo', 'cpf', 'email', 'telefone', 'salario']

for col in colunas_comparar:
    print(f'\n📌 {col.upper()}')
    print(f'   Original:    {df_original[col].iloc[0]}')
    print(f'   Anonimizado: {df_anonimizado[col].iloc[0]}')

In [ ]:
# Visualização tabular
comparacao = pd.DataFrame({
    'Coluna': colunas_comparar,
    'Original': [str(df_original[col].iloc[0])[:30] for col in colunas_comparar],
    'Anonimizado': [str(df_anonimizado[col].iloc[0])[:30] for col in colunas_comparar],
    'Método': [config_anonimizacao.get(col, {}).get('method', '-') for col in colunas_comparar]
})

fig = go.Figure(data=[go.Table(
    header=dict(
        values=['<b>Coluna</b>', '<b>Original</b>', '<b>Anonimizado</b>', '<b>Método</b>'],
        fill_color=CORES['primaria'],
        font=dict(color='white', size=13),
        align='left',
        height=35
    ),
    cells=dict(
        values=[comparacao[col] for col in comparacao.columns],
        fill_color=[['#f8f9fa', 'white'] * 3],
        font=dict(size=12),
        align='left',
        height=30
    )
)])

fig.update_layout(
    title={'text': '🔒 Comparação: Dados Originais vs Anonimizados', 'x': 0.5, 'font': {'size': 18}},
    height=300
)

fig.show()

## 4. Verificação Pós-Anonimização

Vamos executar o scanner novamente para verificar se os dados ainda contêm PIIs detectáveis.

In [ ]:
# Scan pós-anonimização
resultado_pos = scanner.scan(df_anonimizado, source_name='dados_anonimizados.csv')

# Comparar resultados
print('=' * 70)
print('COMPARAÇÃO: ANTES vs DEPOIS DA ANONIMIZAÇÃO')
print('=' * 70)

metricas = pd.DataFrame({
    'Métrica': ['PIIs Detectados', 'Risco Crítico', 'Risco Alto', 'Risco Médio', 'Risco Baixo'],
    'Antes': [
        len(resultado.pii_found),
        resultado.risk_summary.get('critico', 0),
        resultado.risk_summary.get('alto', 0),
        resultado.risk_summary.get('medio', 0),
        resultado.risk_summary.get('baixo', 0)
    ],
    'Depois': [
        len(resultado_pos.pii_found),
        resultado_pos.risk_summary.get('critico', 0),
        resultado_pos.risk_summary.get('alto', 0),
        resultado_pos.risk_summary.get('medio', 0),
        resultado_pos.risk_summary.get('baixo', 0)
    ]
})

metricas['Redução'] = metricas['Antes'] - metricas['Depois']
metricas['Redução %'] = ((metricas['Antes'] - metricas['Depois']) / metricas['Antes'].replace(0, 1) * 100).round(1)

print(metricas.to_string(index=False))

In [ ]:
# Gráfico comparativo
fig = go.Figure()

categorias = ['Crítico', 'Alto', 'Médio', 'Baixo']
valores_antes = [resultado.risk_summary.get(c.lower(), 0) for c in ['critico', 'alto', 'medio', 'baixo']]
valores_depois = [resultado_pos.risk_summary.get(c.lower(), 0) for c in ['critico', 'alto', 'medio', 'baixo']]

fig.add_trace(go.Bar(
    name='Antes',
    x=categorias,
    y=valores_antes,
    marker_color=CORES['secundaria'],
    text=valores_antes,
    textposition='outside'
))

fig.add_trace(go.Bar(
    name='Depois',
    x=categorias,
    y=valores_depois,
    marker_color=CORES['primaria'],
    text=valores_depois,
    textposition='outside'
))

fig.update_layout(
    title={'text': '📊 Redução de Risco Após Anonimização', 'x': 0.5, 'font': {'size': 18}},
    xaxis_title='Nível de Risco',
    yaxis_title='Quantidade de Colunas',
    barmode='group',
    height=400
)

fig.show()

## 5. Sumário Executivo

In [ ]:
# Calcular redução total de risco
pii_antes = len(resultado.pii_found)
pii_depois = len(resultado_pos.pii_found)
reducao_total = ((pii_antes - pii_depois) / pii_antes * 100) if pii_antes > 0 else 100

print('=' * 70)
print('SUMÁRIO EXECUTIVO - AUDITORIA LGPD')
print('=' * 70)

print(f'''
📄 DATASET ANALISADO
   Registros: {len(df_original):,}
   Colunas: {len(df_original.columns)}
   Fonte: dados_clientes.csv

🔍 SCAN DE DADOS PESSOAIS
   PIIs detectados (antes): {pii_antes}
   Riscos críticos: {resultado.risk_summary.get('critico', 0)}
   Riscos altos: {resultado.risk_summary.get('alto', 0)}

🔒 ANONIMIZAÇÃO APLICADA
   Colunas processadas: {len(config_anonimizacao)}
   Métodos utilizados: {len(set(c['method'] for c in config_anonimizacao.values()))}

📊 RESULTADO
   PIIs após anonimização: {pii_depois}
   Redução de risco: {reducao_total:.1f}%

✅ STATUS: {'CONFORME' if reducao_total >= 80 else 'REQUER ATENÇÃO'}
''')

print('=' * 70)
print('Referências LGPD aplicadas:')
print('   • Art. 5º, II - Dado anonimizado')
print('   • Art. 12 - Anonimização como alternativa')
print('   • Art. 6º, III - Princípio da necessidade')
print('=' * 70)

## 6. Exportar Dados Anonimizados

In [ ]:
# Salvar dados anonimizados
import os

# Criar diretório de saída (funciona tanto local quanto no Colab)
output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)

arquivo_saida = os.path.join(output_dir, 'dados_anonimizados.csv')
df_anonimizado.to_csv(arquivo_saida, index=False, encoding='utf-8-sig')

tamanho_kb = os.path.getsize(arquivo_saida) / 1024

print(f'✅ Dados anonimizados salvos em: {arquivo_saida}')
print(f'   Registros: {len(df_anonimizado):,}')
print(f'   Tamanho: {tamanho_kb:.1f} KB')

# No Colab, oferecer download
try:
    from google.colab import files
    files.download(arquivo_saida)
    print('📥 Download iniciado automaticamente!')
except:
    pass  # Não está no Colab

---

## 🚀 Próximos Passos

Este notebook demonstrou o fluxo completo de auditoria e anonimização LGPD.

**Para usar o sistema via CLI:**

```bash
# Scan de arquivo
python main.py scan dados.csv --report

# Anonimização
python main.py anonymize dados.csv --config config.json
```

**Recursos adicionais:**
- Relatório HTML detalhado
- Configuração JSON para anonimização
- Logs de auditoria

---

*Desenvolvido como parte do Portfólio de Data Science*